### 导入需要的包

In [1]:
!apt update
!apt install chromium-chromedriver

# 检查环境中是否有selenium/webdriver_manager，若无，则直接安装
# 安装依赖
!pip install selenium
!pip install webdriver_manager

# https://developers.google.com/drive/api/v3/quickstart/python
!pip install pygsheets
!pip install -U google-api-python-client google-auth-httplib2 google-auth-oauthlib


Unable to locate an executable at "/Library/Java/JavaVirtualMachines/jdk1.8.0_211.jdk/Contents/Home/bin/apt" (-1)
Unable to locate an executable at "/Library/Java/JavaVirtualMachines/jdk1.8.0_211.jdk/Contents/Home/bin/apt" (-1)
     |████████████████████████████████| 61 kB 61 kB/s  eta 0:00:01
Requirement already up-to-date: google-auth-httplib2 in /opt/anaconda3/lib/python3.7/site-packages (0.0.4)
Requirement already up-to-date: google-auth-oauthlib in /opt/anaconda3/lib/python3.7/site-packages (0.4.2)


  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.5
    Uninstalling google-api-python-client-1.12.5:
      Successfully uninstalled google-api-python-client-1.12.5


In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
# 这一块代码，大家可以不需要了解意思，只要 copy 过来就好了
from __future__ import print_function
from pathlib import Path
import os
import io
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient import errors
from googleapiclient.http import MediaIoBaseDownload

import pygsheets
import pandas as pd

# If modifying these scopes, delete the file token.pickle.
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# add profile just for support different google accounts
def get_creds(profile=""):
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    home = str(Path.home())
    google_helper = os.path.join(home,".google_helper")
    print(google_helper)
    Path(google_helper).mkdir(parents=True, exist_ok=True)
    
    if len(profile)>0:
        token_path = os.path.join(home,".google_helper","{}_token.pickle".format(profile))
        credential_path = os.path.join(home,".google_helper","{}_credentials.json".format(profile))
    else:
        token_path = os.path.join(home,".google_helper","token.pickle")
        credential_path = os.path.join(home,".google_helper","credentials.json")

    if not os.path.exists(credential_path):
        print("Google 授权文件:{}, 不存在!".format(credential_path))
        print("请把 credentials.json 放在 {} 这个目录下 :".format(google_helper))
        return None
        
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)
    
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credential_path, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_path, 'wb') as token:
            pickle.dump(creds, token)
    return creds
# 得到 google drive/sheet 的权限
creds = get_creds()
if creds is None:
    print("请配好权限！")

/Users/anker/.google_helper


In [4]:
## 定义访问的链接
HOME_URL = 'https://www.naver.com/'

In [5]:
chrome_options = webdriver.ChromeOptions()
###chrome_options.add_argument('--user-data-dir=chrome-data') # 使用chrome缓存的用户信息

# 其它使用ChromeDriverManager(要求系统中有chrome浏览器)
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

#driver.maximize_window()
# 全屏显示
#driver.fullscreen_window()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/anker/.wdm/drivers/chromedriver/mac64/87.0.4280.20/chromedriver] found in cache


## 主程序

In [6]:
## 第一步访问链接
driver.get(HOME_URL)

In [ ]:
## 关掉弹出来的广告
ad_path = '//*[@id="widgets-view-email-modal-mount"]/div/div/div[1]/div/div/div/div/button'
driver.find_element_by_xpath(ad_path).click()

In [9]:
## 定位到搜索框  //*[@id="query"]
search_input_path = '//*[@id="query"]'
search_input = driver.find_element_by_xpath(search_input_path)

In [10]:
## 输入要搜索的关键词
## 这里输入关键词 可以考虑去读取表格 然后循环查询不同的关键词写入不同的sheet

search_input.send_keys('bluetooth speaker')


In [11]:
## 点击搜索 //*[@id="search_btn"]/div[2]/div[1]/div/div[2]/div/div[1]/div/div/form/button[2]
search_click_path = '//*[@id="search_btn"]/div[2]/div[1]/div/div[2]/div/div[1]/div/div/form/button[2]'
driver.find_element_by_xpath(search_click_path).click()

In [13]:
## 将抓取到的信息封装成一个字典
def get_sku_item_info(element_item):
    sku_title = element_item.find_element_by_xpath(".//*[@class='sku-header']").text
    sku = element_item.find_element_by_xpath(".//*[@class='sku-value']").text
    price = element_item.find_element_by_xpath(".//*[@class='priceView-hero-price priceView-customer-price']/span[1]").text
    ##  可以在这里继续加数据 抓取剩余的数据
    return {"sku_title":sku_title,"sku":sku,"price":price}

In [17]:
## 没有翻页的

## 嗯从这里开始我也不知道我在干啥 //*[@data-email="'+handover_people+'"]
## priceView-hero-price priceView-customer-price
## //*[@id="pricing-price-45499384"]/div/div/div/div/div[2]/div/div/div/span[1]
## class="sku-header"
sku_list = []
##  抓取之前记得处理等待时间 如果网页还没加载完成 会报错的 
result_element =  driver.find_elements_by_xpath('//*[@id="main-results"]/ol/li')
for element_item in result_element:
    ## 注意try except的用法
    try:
        sku_list.append(get_sku_item_info(element_item))
    except:
        print("Main Results Continued")

## 利用字典 拼接df数据 https://blog.csdn.net/u010199356/article/details/85697860 学会查资料
df = pd.DataFrame(sku_list, columns = ["sku_title", "sku", "price"])

# 验证登录
gc = pygsheets.authorize(custom_credentials=creds)

# 创建一个 Google Sheet
sheet_table = gc.create('skuItemInfo')
sheet = sheet_table.worksheet_by_title('Sheet1')
sheet.set_dataframe(df,(1,1))

    

Main Results Continued
Main Results Continued
Main Results Continued


In [ ]:
## 翻页的 如果有翻页可以考虑

    